<a href="https://colab.research.google.com/github/99bbarton/TaustarToTauTauZ/blob/main/EvtVisualizationEventVisualizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Imports

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np
import pandas as pd
from IPython.display import clear_output

# Source Code

## Utilities

### Graphics Utilities

In [ ]:
colDict = {"tau(h)": "#c22c0b", "tau(e)" :"#13500b", "tau(mu)" : "#08357f",
           "Z(AK8)":"#521077", "Z_AK4SJ": "#93139b","Z(ee)":"#85ba89", "Z(mumu)":"#2275c0",
           "Z_pfc(211)":"#fee302", "Z_pfc(-211)":"#fee302", "Z_pfc(130)":"#fee302", "Z_pfc(22)":"#fe8002", "AK4": "#8d0e03", "Z_reCl":"#5d0a03"}

nameDict = {"tau(h)": r"$\tau_h$", "tau(e)" :r"$\tau_e$", "tau(mu)" : r"$\tau_{\mu}$",
           "Z(AK8)":r"$Z\rightarrow AK8$", "Z(ee)": r"$Z\rightarrow ee$", "Z(mumu)":r"$Z\rightarrow\mu\mu$", "Z_AK4SJ": "AK4 Subjet",
           "Z_pfc(211)":r"PFC $\pi^+$", "Z_pfc(-211)":r"PFC $\pi^-$", "Z_pfc(130)":r"PFC $K^0_L$", "Z_pfc(22)":r"PFC $\gamma$", "AK4": "Re-clust. AK4", "Z_reCl": "Re-clust. Z"}

def displayColors(color_dict):
    fig, ax = plt.subplots(figsize=(10, 4), layout="constrained")

    for idx, (key, hex_code) in enumerate(color_dict.items()):

        ax.plot([0.18, 0.58], [(1-idx)+1, (1-idx)+1], color=hex_code, linewidth=3)
        ax.add_patch(patches.Rectangle((0.6, ((1 - idx) - 0.25)+1), 0.5, 0.5, facecolor=hex_code))
        ax.text(0.1, ((1 - idx) +1), nameDict[key], va='center', ha='center', fontsize=12)

    ax.set_title("Particle Rendering Colors")
    ax.axis('off')
    plt.show()


def plotCone(ax, px, py, pz, coneDia, color, alpha=0.2, nTris=50):
    height = np.sqrt(px**2 + py**2 + pz**2)
    radius = height * coneDia/2.0

    theta = np.linspace(0, 2*np.pi, nTris)
    x_base = radius * np.cos(theta) + px
    y_base = radius * np.sin(theta) + py
    z_base = np.full_like(theta, pz)

    ax.plot_trisurf([px] + list(x_base), [py] + list(y_base), [pz] + list(z_base), color=color, alpha=alpha)

    for xb, yb, zb in zip(x_base, y_base, z_base):
        verts = [[(0, 0, 0), (px, py, pz), (xb, yb, zb)]]
        ax.add_collection3d(Poly3DCollection(verts, color=color, alpha=alpha, edgecolor='none'))


In [ ]:
displayColors(colDict)

### Physics Utilities

In [ ]:
def etaToTheta(eta):
    return 2*np.arctan(np.exp(-eta))

def thetaToEta(theta):
    return -1 * np.log(np.tan(theta/2.0))

def p3ToEPEtaPhi(px, py, pz, m):
    p = np.sqrt(px**2 + py**2 + pz**2)
    eta = 0.5 * np.log((p + pz) / (p - pz))
    phi = np.arctan2(py, px)
    E = np.sqrt(p**2 + m**2)
    return E, p, eta, phi

def distance(row1, row2, p, R):
   dR = np.sqrt((row1["eta"] - row2["eta"])**2 + (row1["phi"] - row2["phi"])**2)
   pWeight = min(row1["p[GeV]"], row2["p[GeV]"])**(2*p)
   return pWeight * dR / R

## Visualizer Class

In [ ]:
class Visualizer:
    data = None
    nEvents = 0
    currEvtNum = 0
    prevBoost = np.array([0,0,0])

    def __init__(self):
        pass

    # ---------------------------------------------------------------------------------------------------------------------------- #

    def readCSV(self, filepath):
        self.data = pd.read_csv(filepath, usecols=['event', 'particle', 'px[GeV]', 'py[GeV]', 'pz[GeV]', 'mass[GeV]',
       'd0[cm]', 'dz[cm]', 'isTrack', 'coneR'],dtype = {'event':np.int32, 'particle':str, 'px[GeV]':np.float32, 'py[GeV]':np.float32, 'pz[GeV]':np.float32, 'mass[GeV]':np.float32,
       'd0[cm]':np.float32, 'dz[cm]':np.float32, 'isTrack':bool, 'coneR':np.float32} )
        self.nEvents = self.data["event"].max() + 1

    # ---------------------------------------------------------------------------------------------------------------------------- #

    def readDF(self, df):
        self.data = df
        self.nEvents = self.data["event"].max() + 1

    # ---------------------------------------------------------------------------------------------------------------------------- #

    def getEvent(self, eventNum=-1):
        if eventNum < 0 or eventNum >= self.nEvents:
            eventNum = self.currEvtNum
        else:
            self.currEvtNum = eventNum
        return self.data[self.data["event"]==eventNum]

    # ---------------------------------------------------------------------------------------------------------------------------- #

    def filterEvent(self, event, exclList):
        filteredEvent = event
        for criterion in exclList:
            if criterion.endswith("*"):
                filteredEvent = filteredEvent.loc[~event["particle"].str.startswith(criterion[:-1])]
            elif criterion.endswith("*"):
                filteredEvent = filteredEvent.loc[~event["particle"].str.endswith(criterion[1:])]
            else:
                filteredEvent = filteredEvent.loc[event["particle"] != criterion]

        return filteredEvent

    # ---------------------------------------------------------------------------------------------------------------------------- #

    def boostEvent(self, event, beta):

        boosted = event.copy(deep=True)

        if "p[GeV]" not in boosted.columns:
            boosted = self.calcEMomEtaPhi(boosted)

        betaSqr = np.dot(beta, beta)
        gamma = 1.0 / np.sqrt(1.0 - betaSqr)
        if betaSqr > 0:
            gamma2 = (gamma - 1.0) / betaSqr
        else:
            gamma2 = 0.0

        for rowN, row in event.iterrows():
            p3 = np.array( [row["px[GeV]"], row["py[GeV]"], row["pz[GeV]"]])
            bp = np.dot(beta, p3)

            newE = gamma * (row["E[GeV]"] - bp)
            newP3 = p3 + (gamma2 * bp * beta) - (gamma * beta * row["E[GeV]"])

            boosted.at[rowN, "px[GeV]"] = np.float32(newP3[0])
            boosted.at[rowN, "py[GeV]"] = np.float32(newP3[1])
            boosted.at[rowN, "pz[GeV]"] = np.float32(newP3[2])
            boosted.at[rowN, "E[GeV]"] =  np.float32(newE)

        boosted = self.calcEMomEtaPhi(boosted)

        return boosted

    # ---------------------------------------------------------------------------------------------------------------------------- #

    def boostTo(self, partName, event=None, ptRank="DOM", exclFilter = [],):
        if partName not in ["Z", "Z(AK8)", "Z(ee)", "Z(mumu)", "Z_AK4SJ", "SJ", "SJTOT", "PFTOT"]:
            print("\n\nERROR: Unrecognized particle given as target to boostTo!\n\n")
            return None

        if event is None:
            event = self.getEvent()

        if len(exclFilter) > 0:
            event = self.filterEvent(event, exclList=exclFilter)

        if "p[GeV]" not in event.columns:
            event = self.calcEMomEtaPhi(event)

        if partName == "Z":
            if "Z(AK8)" in event["particle"].values:
                partName = "Z(AK8)"
            elif "Z(ee)" in event["particle"].values:
                partName = "Z(ee)"
            elif "Z(mumu)" in event["particle"].values:
                partName = "Z(mumu)"

        if partName in ["Z(AK8)", "Z(ee)", "Z(mumu)"]:
            row = event[event["particle"]==partName]
            boostVec = np.array([-row.iloc[0]["px[GeV]"], -row.iloc[0]["py[GeV]"], -row.iloc[0]["pz[GeV]"]])
            boostVec = boostVec / row.iloc[0]["E[GeV]"]
        elif partName in ["Z_AK4SJ", "SJ"]:
            subJets = event[event["particle"]=="Z_AK4SJ"]
            if ptRank == "SUBDOM":
                row = subJets.loc[subJets["p[GeV]"].idxmin()]
            else:
                row = subJets.loc[subJets["p[GeV]"].idxmax()]
            boostVec = np.array([-row["px[GeV]"], -row["py[GeV]"], -row["pz[GeV]"]])
            boostVec = boostVec / row["E[GeV]"]
        elif partName == "SJTOT":
            subJets = event[event["particle"]=="Z_AK4SJ"]
            boostVec = np.array([-subJets["px[GeV]"].sum(), -subJets["py[GeV]"].sum(), -subJets["pz[GeV]"].sum()])
            boostVec = boostVec / subJets["E[GeV]"].sum()
        elif partName == "PFTOT":
            pfcs = event[event["particle"].str.startswith("Z_pfc")]
            boostVec = np.array([-pfcs["px[GeV]"].sum(), -pfcs["py[GeV]"].sum(), -pfcs["pz[GeV]"].sum()])
            boostVec = boostVec / pfcs["E[GeV]"].sum()

        if boostVec is not None:
            boostVec = -1 * boostVec
            self.prevBoost = boostVec
            boostEvt = self.boostEvent(event, boostVec)

        return boostEvt

    # ---------------------------------------------------------------------------------------------------------------------------- #

    def unBoost(self, event=None):
        if event is None:
            event = self.getEvent()

        self.prevBoost = -1 * self.prevBoost
        return self.boostEvent(event, self.prevBoost)

    # ---------------------------------------------------------------------------------------------------------------------------- #

    def calcEMomEtaPhi(self, event=None):
        if event is None:
            self.data["E[GeV]"], self.data["p[GeV]"], self.data["eta"], self.data["phi"] = p3ToEPEtaPhi(self.data["px[GeV]"], self.data["py[GeV]"], self.data["pz[GeV]"], self.data["mass[GeV]"])
            return 0
        else:
            eventCopy = event.copy(deep=True)
            eventCopy["E[GeV]"], eventCopy["p[GeV]"], eventCopy["eta"], eventCopy["phi"] = p3ToEPEtaPhi(event["px[GeV]"], event["py[GeV]"], event["pz[GeV]"], event["mass[GeV]"])
            return eventCopy

    # ---------------------------------------------------------------------------------------------------------------------------- #

    def recluster(self, event, p=0, R=0.4, momCut = -1, nPFCpJCut = -1):

        if "p[GeV]" not in event.columns:
            event = self.calcEMomEtaPhi(event)

        reclustDF = event.copy(deep=True)
        reclustDF.drop(reclustDF.index)

        event = event[event["particle"].str.startswith("Z_pfc")]

        nPFCsperJet = np.zeros(event.shape[0])
        nPFCsperJet += 1
        nPFCsperJet = nPFCsperJet.tolist()
        jetToPFCN = {}

        while event.shape[0] > 0:
            minDist = 9999999
            minPairIdxs = None
            minBDist = 9999999
            minBDistIdx = -1

            for rowN1, row1 in event.iterrows():
                for rowN2, row2 in event.iterrows():
                    if rowN1 == rowN2:
                        continue
                    dist = distance(row1, row2, p, R)
                    if dist < minDist:
                        minDist = dist
                        minPairIdxs = (rowN1, rowN2)

                bDist = row1["p[GeV]"]**(2*p)
                if bDist < minBDist:
                    minBDist = bDist
                    minBDistIdx = rowN1

            if minDist < minBDist:
                idx1 = minPairIdxs[0]
                idx2 = minPairIdxs[1]

                event.at[idx1, "px[GeV]"] += event.at[idx2, "px[GeV]"]
                event.at[idx1, "py[GeV]"] += event.at[idx2, "py[GeV]"]
                event.at[idx1, "pz[GeV]"] += event.at[idx2, "pz[GeV]"]
                event.at[idx1, "E[GeV]"] += event.at[idx2, "E[GeV]"]
                event.at[idx1, "coneR"] = np.float32(R)
                event.at[idx1, "isTrack"] = False
                event.at[idx1, "particle"] = "AK4"
                event = event.drop(idx2)
                event.reset_index(drop=True, inplace=True)

                nPFCsperJet[idx1] += nPFCsperJet[idx2]
                del nPFCsperJet[idx2]
            else:

                event.at[minBDistIdx, "coneR"] = np.float32(R)
                event.at[minBDistIdx, "isTrack"] = False
                event.at[minBDistIdx, "particle"] = "AK4"
                if nPFCpJCut > 0:
                    if nPFCsperJet[minBDistIdx] < nPFCpJCut:
                        event = event.drop(minBDistIdx)
                        event.reset_index(drop=True, inplace=True)
                        del nPFCsperJet[minBDistIdx]
                        continue

                reclustDF = pd.concat([reclustDF, event.iloc[[minBDistIdx]]], ignore_index=True)
                event = event.drop(minBDistIdx)
                del nPFCsperJet[minBDistIdx]
                event.reset_index(drop=True, inplace=True)

        reclustDF = self.calcEMomEtaPhi(reclustDF)

        if momCut > 0:
            reclustDF = pd.concat([reclustDF[(reclustDF["particle"] != "AK4")], reclustDF[(reclustDF["particle"] == "AK4") & (reclustDF["p[GeV]"] > momCut)]], ignore_index=True)

        ak4_jets = reclustDF[reclustDF["particle"] == "AK4"]
        if not ak4_jets.empty:

            total_px = ak4_jets["px[GeV]"].sum()
            total_py = ak4_jets["py[GeV]"].sum()
            total_pz = ak4_jets["pz[GeV]"].sum()
            total_E = ak4_jets["E[GeV]"].sum()

            total_mass = np.sqrt(max(total_E**2 - (total_px**2 + total_py**2 + total_pz**2), 0))

            z_recl = {
                "event" : reclustDF.at[1, "event"],
                "px[GeV]": total_px,
                "py[GeV]": total_py,
                "pz[GeV]": total_pz,
                "E[GeV]": total_E,
                "mass[GeV]": total_mass,
                "particle": "Z_reCl",
                "coneR": 0.8,
                "isTrack": False,
                "d0[cm]" : reclustDF.at[1, "d0[cm]"],
                "dz[cm]" : reclustDF.at[1, "dz[cm]"]
            }

            reclustDF = pd.concat([reclustDF, pd.DataFrame([z_recl])], ignore_index=True)
            reclustDF = self.calcEMomEtaPhi(reclustDF)

        reclustDF.reset_index(drop=True, inplace=True)

        return reclustDF


    # ---------------------------------------------------------------------------------------------------------------------------- #

    def addBack(self, event, addList):
        for part in addList:

            event = pd.concat([event, self.data[(self.data["event"]==event.iloc[0]["event"]) & (self.data["particle"].str.startswith(part))]], ignore_index=True)

        return self.calcEMomEtaPhi(event)

    # ---------------------------------------------------------------------------------------------------------------------------- #

    def vis_rz(self, event, boostLabel = "", etaLines=True, axis=True, alpha=0.3):
        fig, ax = plt.subplots(figsize=(8, 8))

        ax.set_xlim(-1.2, 1.2)
        ax.set_ylim(-1.2, 1.2)
        ax.set_aspect('equal')
        ax.set_axis_off()
        plt.grid(False)
        titleStr = "Z-R View: Event #" + str(event.iloc[0]["event"])
        if boostLabel != "":
            titleStr += " Boosted to " + boostLabel + " Frame"
        ax.set_title(titleStr)

        if axis:
            ax.plot([-1.2, 1.2], [0, 0], color="lightgrey" )
            ax.text(1.2, 0.05, "+Z", fontsize=10, ha='center', va='center', color='lightgray', zorder=0)
            ax.text(-1.2, 0.05, "-Z", fontsize=10, ha='center', va='center', color='lightgray', zorder=0)

        if etaLines:
            etas = np.arange(-2.5, 3.0, 0.5)
            thetas = etaToTheta(etas)
            for eta, theta in zip(-etas, thetas):
                x, y = np.cos(theta), np.sin(theta)
                ax.plot([0, x], [0, y], color='lightgray')
                ax.plot([0, x], [0, -y], color='lightgray')
                ax.text(x * 1.2, y * 1.1, f"$\eta=${-eta:.1f}", fontsize=10, ha='center', va='center', color='lightgray', zorder=0)
                ax.text(x * 1.2, -y * 1.1, f"$\eta=${-eta:.1f}", fontsize=10, ha='center', va='center', color='lightgray', zorder=0)

        if "p[GeV]" not in event.columns:
            event = self.calcEMomEtaPhi(event)

        legEntries = {}

        for rowN, row in event.iterrows():
            color = colDict[row["particle"]]
            label = nameDict[row["particle"]]

            if row["p[GeV]"] < 1:
                continue
            pT = np.sqrt((row["p[GeV]"]**2) - (row["pz[GeV]"])**2)
            if row["isTrack"]:
                ax.plot([0, row["pz[GeV]"]], [0, pT], color=color)
            else:
                dR =  row["coneR"] / 2.0
                thetaMinus = etaToTheta(row["eta"] - dR)
                thetaPlus =  etaToTheta(row["eta"] + dR)
                triangle = plt.Polygon([(0, 0), (row["p[GeV]"]*np.cos(thetaMinus), row["p[GeV]"]*np.sin(thetaMinus)), (row["p[GeV]"]*np.cos(thetaPlus), row["p[GeV]"]*np.sin(thetaPlus))], color=color, alpha=alpha)
                ax.add_patch(triangle)

            if label not in legEntries:
                legEntries[label] = patches.Patch(color=color, label=label)

        ax.legend(handles=legEntries.values(), loc='upper right', fontsize=10)

        plt.show()

    # ---------------------------------------------------------------------------------------------------------------------------- #

    def vis_xy(self, event,boostLabel="", alpha=0.3, axis=True):
        fig, ax = plt.subplots(figsize=(8, 8))

        ax.set_xlim(-1.2, 1.2)
        ax.set_ylim(-1.2, 1.2)
        ax.set_aspect('equal')
        ax.set_axis_off()
        plt.grid(False)
        titleStr = "X-Y View: Event #" + str(event.iloc[0]["event"])
        if boostLabel != "":
            titleStr += " Boosted to " + boostLabel + " Frame"
        ax.set_title(titleStr)

        if axis:
            ax.plot([-1.2, 1.2], [0, 0], color="lightgrey" )
            ax.plot([0, 0], [-1.2, 1.2], color="lightgrey" )

        if "p[GeV]" not in event.columns:
            event = self.calcEMomEtaPhi(event)

        legEntries = {}

        for rowN, row in event.iterrows():
            if row["p[GeV]"] < 1:
                continue

            color = colDict[row["particle"]]
            label = nameDict[row["particle"]]

            if row["isTrack"]:
                ax.plot([0, row["px[GeV]"]], [0, row["py[GeV]"]], color=colDict[row["particle"]])
            else:
                dR =  row["coneR"] / 2.0
                phiMinus = row["phi"] - dR
                phiPlus = row["phi"] + dR
                pT = np.sqrt(row["p[GeV]"]**2 -row["pz[GeV]"]**2)
                triangle = plt.Polygon([(0, 0),(pT*np.cos(phiMinus), pT*np.sin(phiMinus)) , (pT*np.cos(phiPlus), pT*np.sin(phiPlus))], color=colDict[row["particle"]], alpha=alpha)
                ax.add_patch(triangle)

            if label not in legEntries:
                legEntries[label] = patches.Patch(color=color, label=label)

        ax.legend(handles=legEntries.values(), loc='upper right', fontsize=10)

        plt.show()

    # ---------------------------------------------------------------------------------------------------------------------------- #

    def vis_3D(self, event, boostLabel="", nTris = 50, alpha=0.1):
        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(111, projection='3d')
        titleStr = "3D Visualization: Event #" + str(event.iloc[0]["event"])
        if boostLabel != "":
            titleStr += " Boosted to " + boostLabel + " Frame"
        ax.set_title(titleStr)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_zticklabels([])
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_zlabel("Z")

        legEntries = {}

        if "p[GeV]" not in event.columns:
            event = self.calcEMomEtaPhi(event)

        for rowN, row in event.iterrows():
            color = colDict[row["particle"]]
            label = nameDict[row["particle"]]

            if row["p[GeV]"] < 1:
                continue

            if row["isTrack"]:
                ax.plot([0, row["px[GeV]"]], [0, row["py[GeV]"]], [0, row["pz[GeV]"]], color=colDict[row["particle"]])
            else:
                plotCone(ax, row["px[GeV]"], row["py[GeV]"], row["pz[GeV]"], row["coneR"],  colDict[row["particle"]], alpha=alpha, nTris=nTris)

            if label not in legEntries:
                legEntries[label] = patches.Patch(color=color, label=label)

        ax.legend(handles=legEntries.values(), loc='upper right', fontsize=10)

        plt.show()

    # ---------------------------------------------------------------------------------------------------------------------------- #

    def iterEvents(self, startEvt = 0, prnt = False, view2D = True, view3D = True, boostTo="",  exclList=[], etaLines=True, axis=True, alpha2D=0.2, alpha3D=0.1, nTris=50):
        for evtN in range( startEvt, self.nEvents):
            event = self.getEvent(evtN)

            if len(boostTo) > 0:
                event = self.boostTo(partName=boostTo, event=event)

            event = self.filterEvent(event, exclList=exclList)

            if prnt:
                print(event)
            if view2D:
                self.vis_rz(event, etaLines=etaLines, axis=axis, alpha=alpha2D, boostLabel=boostTo)
                self.vis_xy(event, axis=axis, alpha=alpha2D, boostLabel=boostTo)
            if view3D:
                self.vis_3D(event, nTris=nTris, alpha=alpha3D, boostLabel=boostTo)

            resp = input('Hit ENTER to continue to the next event or "D" or "DONE" to stop iterating: ')
            clear_output(wait=True)
            if resp == "D" or resp == "DONE":
                break

    # ---------------------------------------------------------------------------------------------------------------------------- #




# Testing

In [ ]:
vis = Visualizer()
vis.readCSV("eventVisualizationInput.csv")
evt = vis.calcEMomEtaPhi(vis.getEvent(0))
#evt.head()

In [ ]:
vis.vis_xy(evt)
vis.vis_rz(evt, alpha=0.2)
vis.vis_3D(evt, nTris=50, alpha=0.05)

In [ ]:
targ = "Z(AK8)"
boostEvt = vis.boostTo(targ, exclFilter=["tau*"])
vis.vis_xy(boostEvt, boostLabel=targ)
vis.vis_rz(boostEvt, boostLabel=targ)
vis.vis_3D(boostEvt, boostLabel=targ)
#boostEvt

In [ ]:
vis.iterEvents(startEvt=0, prnt=False, view2D=True, view3D=True, boostTo="Z", exclList=["tau*"])

# Reclustering

In [ ]:
targ = "Z(AK8)"
boostEvt = vis.boostTo(targ, event=vis.getEvent(0), exclFilter=["tau*"])
origBoost = vis.prevBoost

reclusEvt = vis.recluster(boostEvt, p=0, R=0.4, nPFCpJCut=2)
onlyRecl = vis.filterEvent(reclusEvt, exclList=["Z(*", "Z_AK4*"])
noReClZ = vis.filterEvent(onlyRecl, exclList=["Z_reCl*"])

vis.vis_xy(noReClZ, boostLabel=targ)
vis.vis_rz(noReClZ, boostLabel=targ)
vis.vis_3D(noReClZ, boostLabel=targ)

In [ ]:
unBoosted = vis.unBoost(onlyRecl)
unBoosted = vis.addBack(unBoosted, "tau")

vis.vis_xy(unBoosted)
vis.vis_rz(unBoosted)
vis.vis_3D(unBoosted)

In [ ]:
print(evt[evt["particle"]=="Z(AK8)"])
print(unBoosted[unBoosted["particle"]=="Z_reCl"])